<a href="https://colab.research.google.com/github/johnnyff/bigcontest2021/blob/main/danm_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import torch
import datetime as dt
import torch.nn as nn

warnings.filterwarnings(action='ignore') 

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/bigcontest/data/bigcontest_dataset.xlsx', header = 1)
df

In [ ]:
columns = ['num','year','month','day','hour','target',
           'average_rain_1','a_rain_1','b_rain_1','c_rain_1','d_rain_1','e_level_1','d_level_1',
           'average_rain_2','a_rain_2','b_rain_2','c_rain_2','d_rain_2','e_level_2','d_level_2',
           'average_rain_3','a_rain_3','b_rain_3','c_rain_3','d_rain_3','e_level_3','d_level_3',
           'average_rain_4','a_rain_4','b_rain_4','c_rain_4','d_rain_4','e_level_4','d_level_4',
           'average_rain_5','a_rain_5','b_rain_5','c_rain_5','d_rain_5','e_level_5','d_level_5',
           'average_rain_6','a_rain_6','b_rain_6','c_rain_6','d_rain_6','e_level_6','d_level_6'
           ]

In [ ]:
df.columns =columns


In [ ]:
df['num'].value_counts()

In [ ]:
df.drop(['year','month','day','hour'],axis =1 , inplace = True)
df

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.duca.manual_seed_all(777)

In [ ]:
import torch
from torch.nn import Module, LSTM, Linear
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable

In [ ]:
train_data = df[:-160]
train_data[train_data['num']==2]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_data.iloc[:,1:])
scaler

In [ ]:

part = train_data[train_data['num']==1]
part.drop(['num'], axis =1 , inplace =True)
a = scaler.transform(part)
a

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []
    for j in range(1,26):
      part = data[data['num']==j]
      part.drop(['num'], axis =1 , inplace =True)
      # part = scaler.transform(part)
      for i in range(len(part)-seq_length):
      
        _x = part[i:(i+seq_length),1:]
        _y = part[i+seq_length-1,0]
        x.append(_x)
        y.append(_y)
    return np.array(x),np.array(y)

In [ ]:
seq_length = 3
x, y = sliding_windows(train_data, seq_length)




In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1422)
print(X_train.shape, X_test.shape)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
X_train_t = torch.Tensor(X_train)
X_test_t = torch.Tensor(X_test)
y_train_t = torch.Tensor(y_train)
y_test_t = torch.Tensor(y_test)

In [ ]:
batch_size = 1
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), shuffle=True, batch_size=batch_size, num_workers = 3)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), shuffle=False, batch_size=batch_size,num_workers = 3)

In [ ]:
for seq, labels in train_loader:
  print(seq.size())
  print(len(seq))
  y_pred = model(seq)
  print(y_pred)
  # print(loss_function(y_pred, labels).item())
  # print(len(seq))
  break

In [ ]:

class LSTM(nn.Module):
    def __init__(self, input_size=42, hidden_layer_size=120, layer_dim =1, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_size, hidden_layer_size, layer_dim, batch_first = True)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))
        
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, input_seq):

        h0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).requires_grad_()
        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, input_seq.size(0), self.hidden_layer_size).requires_grad_()

        batch_size = input_seq.size(0)
   
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        # print(lstm_out.view(len(input_seq),-1)[:,-1].shape)

        out = self.linear(lstm_out)
        out = out.view(batch_size,-1)
        out = out[:,-1]

        return out


In [ ]:
model = LSTM().to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
model

In [ ]:
epoch_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.9, verbose = True)


In [ ]:
epochs = 150
patience = 30
min_val_loss = 9999
for i in range(epochs):
    mse_train =0 
    for seq, labels in train_loader:
        seq = seq.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        mse_train+= loss.item()*seq.shape[0]
        optimizer.step()
    epoch_scheduler.step()
    with torch.no_grad():
        mse_val = 0
        preds = []
        gt = []
        for seq, labels in test_loader:
            seq = seq.to(device)
            labels = labels.to(device)
            y_pred = model(seq)
            preds.append(y_pred.detach().numpy())
            gt.append(labels.detach().numpy())
            mse_val += loss_function(y_pred, labels).item()*seq.shape[0]
    
    preds = np.concatenate(preds)
    gt = np.concatenate(gt)
    print("iteration : {} , rsme : {}".format(i, mse_val**0.5))
    if min_val_loss > mse_val**0.5:
        min_val_loss = mse_val**0.5
        print("Saving...")
        torch.save(model.state_dict(), "/content/drive/MyDrive/bigcontest/danm_lstm")
        counter = 0
    else: 
        counter += 1
    
    if counter == patience:
        break
    
    

In [ ]:
# train
epochs = 150
patience = 30
min_val_loss = 9999
for i in range(epochs):
    mse_train =0 
    for seq, labels in train_loader:
        seq = seq.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)
        loss = loss_function(y_pred, labels)
        loss.backward()
        mse_train+= loss.item()*seq.shape[0]
        optimizer.step()
    epoch_scheduler.step()
    if i%25 == 1:
        print(f'epoch: {i:3} loss: {loss.item():10.8f}')
    if min_val_loss > mse_train**0.5:
        min_val_loss = mse_train**0.5
        print("Saving...")
        torch.save(model.state_dict(), "/content/drive/MyDrive/bigcontest/danm_lstm")
        counter = 0
    else: 
        counter += 1
    
    if counter == patience:
      break

In [ ]:
with torch.no_grad():
    gt = []
    preds = []
    for seq, labels in test_loader:
        seq = seq.to(device)
        labels = labels.to(device)
        y_pred = model(seq)
        preds.append(y_pred)
        gt.append(labels)


print(len(gt), len(preds))

In [ ]:
gt[0]

In [ ]:
a = scaler.inverse_transform(np.array(a))
a

In [ ]:
a[:,0][3]

In [ ]:
preds = []
model.eval()
for seq, labels in test_loader:
        seq = seq.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, seq_length, model.hidden_layer_size),
                        torch.zeros(1, seq_length, model.hidden_layer_size))

        y_pred = model(seq)
        print(y_pred.
        preds.append(y_pred)

        

In [ ]:
preds

In [ ]:
print(x.shape)
print(y.shape)
print(x[:6])
print(y[0])

In [ ]:
class LSTM(nn.Module):

    def __init__(self):
        super(LSTM, self).__init__()
       
        self.lstm = nn.LSTM(input_size=43, hidden_size=128,
                            num_layers=2, batch_first=True)
        
        self.fc = nn.Linear(128, 1)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        # Propagate input through LSTM
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x 

In [ ]:
num_epochs = 100
learning_rate = 0.01

input_size = 43
hidden_size = 2
num_layers = 2

num_classes = 1

net = LSTM().to(device)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    var_x = Variable(trainX).to(device)
    var_y = Variable(trainY).to(device)
    
    # obtain the loss function
    optimizer.zero_grad()
    out = net(var_x)
    loss = criterion(out, var_y)
    loss.backward()
    
    optimizer.step()
    if epoch % 10 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

plt.plot(testY)
plt.plt(net(textX_tensor).to_numpy())
plt.legend(['original','prediction'])
plt.show

In [ ]:
net = net.eval() 

In [ ]:
with torch.no_grad():
  var_data = Variable(testX)
  pred_test = net(var_data)

In [ ]:
sc.inverse_transform(pred_test.squeeze())

In [ ]:
pred_test.squeeze().shape